In [1]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

z_dim = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
bs = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [8]:
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)
print('mnist_dim', mnist_dim)

mnist_dim 784


In [9]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [10]:
n_epoch = 200
lr = 0.0002 

def train():
    # build network

    G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
    D = Discriminator(mnist_dim).to(device)
    
    # loss
    criterion = nn.BCELoss() 

    # optimizer
    G_optimizer = optim.Adam(G.parameters(), lr = lr)
    D_optimizer = optim.Adam(D.parameters(), lr = lr)
    
    
    def D_train(x):
        #=======================Train the discriminator=======================#
        D.zero_grad()

        # train discriminator on real
        x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
        x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

        D_output = D(x_real)
        D_real_loss = criterion(D_output, y_real)
        D_real_score = D_output

        # train discriminator on facke
        z = Variable(torch.randn(bs, z_dim).to(device))
        x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))

        D_output = D(x_fake)
        D_fake_loss = criterion(D_output, y_fake)
        D_fake_score = D_output

        # gradient backprop & optimize ONLY D's parameters
        D_loss = D_real_loss + D_fake_loss
        D_loss.backward()
        D_optimizer.step()

        return  D_loss.data.item()
    
    
    def G_train(x):
        #=======================Train the generator=======================#
        G.zero_grad()

        z = Variable(torch.randn(bs, z_dim).to(device))
        y = Variable(torch.ones(bs, 1).to(device))

        G_output = G(z)
        D_output = D(G_output)
        G_loss = criterion(D_output, y)

        # gradient backprop & optimize ONLY G's parameters
        G_loss.backward()
        G_optimizer.step()

        return G_loss.data.item()
    
    for epoch in range(1, n_epoch+1):           
        D_losses, G_losses = [], []
        for batch_idx, (x, _) in enumerate(train_loader):
            D_losses.append(D_train(x))
            G_losses.append(G_train(x))

        print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
                (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
        
    return [G, D]

In [11]:
G, D = train()

[1/200]: loss_d: 0.936, loss_g: 2.545
[2/200]: loss_d: 1.145, loss_g: 1.870
[3/200]: loss_d: 1.151, loss_g: 1.556
[4/200]: loss_d: 0.791, loss_g: 2.136
[5/200]: loss_d: 0.620, loss_g: 2.472
[6/200]: loss_d: 0.585, loss_g: 2.662
[7/200]: loss_d: 0.571, loss_g: 2.667
[8/200]: loss_d: 0.642, loss_g: 2.449
[9/200]: loss_d: 0.561, loss_g: 2.735
[10/200]: loss_d: 0.684, loss_g: 2.304
[11/200]: loss_d: 0.713, loss_g: 2.171
[12/200]: loss_d: 0.781, loss_g: 1.960
[13/200]: loss_d: 0.751, loss_g: 2.058
[14/200]: loss_d: 0.689, loss_g: 2.221
[15/200]: loss_d: 0.708, loss_g: 2.139
[16/200]: loss_d: 0.815, loss_g: 1.903
[17/200]: loss_d: 0.799, loss_g: 1.881
[18/200]: loss_d: 0.809, loss_g: 1.847
[19/200]: loss_d: 0.869, loss_g: 1.704
[20/200]: loss_d: 0.896, loss_g: 1.643
[21/200]: loss_d: 0.881, loss_g: 1.679
[22/200]: loss_d: 0.895, loss_g: 1.660
[23/200]: loss_d: 0.922, loss_g: 1.604
[24/200]: loss_d: 0.923, loss_g: 1.597
[25/200]: loss_d: 0.965, loss_g: 1.470
[26/200]: loss_d: 0.978, loss_g: 1

In [12]:
torch.save(G.state_dict(), './state_dicts/generator.pth')

In [13]:
with torch.no_grad():
    test_z = Variable(torch.randn(1, z_dim).to(device))
    G = Generator(g_input_dim=z_dim, g_output_dim=mnist_dim).to(device)
    G.load_state_dict(torch.load('./state_dicts/generator.pth'))
    generated = G(test_z)
    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_gan2.png')

In [4]:
import random

G = Generator(g_input_dim=z_dim, g_output_dim=mnist_dim).to(device)
G.load_state_dict(torch.load('./state_dicts/generator.pth'))

def gen_new_mnist():
    id = random.randint(0, 10**5)
    filename = f'./samples/sample_{id}.png'
    with torch.no_grad():
        test_z = Variable(torch.randn(1, z_dim).to(device))
        generated = G(test_z)
        save_image(generated.view(generated.size(0), 1, 28, 28), filename)
    return filename

In [26]:
from IPython.display import Image
Image(filename=gen_new_mnist()) 